In [8]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.primitives import Estimator
from scipy.optimize import minimize

In [12]:
# Define the cost Hamiltonian for Max-Cut
n_qubits = 6
weights = np.random.uniform(0, 1, size=(n_qubits, n_qubits))  # Random weights
H_C_terms = []
for i in range(n_qubits):
    for j in range(i):
        if weights[i, j] != 0:
            pauli_string = ["I"] * n_qubits
            pauli_string[i] = "Z"
            pauli_string[j] = "Z"
            H_C_terms.append(("".join(pauli_string), -0.5 * weights[i, j]))
H_C = SparsePauliOp.from_list(H_C_terms)

# Define the initial state |+>^n
qc = QuantumCircuit(n_qubits)
qc.h(range(n_qubits))
initial_state = qc

# Define the mixer pool
mixer_pool = []
for i in range(n_qubits):
    pauli_string = ["I"] * n_qubits
    pauli_string[i] = "X"
    mixer_pool.append(SparsePauliOp.from_list([("".join(pauli_string), 1.0)]))

def measure_gradient(operator, ansatz, h_c, estimator):
    """
    Measure the gradient of the cost function with respect to a given operator.
    """
    commutator = (-1j * (operator @ h_c - h_c @ operator)).simplify()
    circuits = [ansatz]
    results = estimator.run([commutator], circuits).result()
    gradients = results.values
    return np.real(gradients[0])

def adapt_qaoa_iteration(ansatz, mixer_pool, h_c, params, estimator):
    """
    Perform one iteration of the ADAPT-QAOA algorithm.
    """
    gradients = [measure_gradient(op, ansatz, h_c, estimator) for op in mixer_pool]
    max_grad_index = np.argmax(np.abs(gradients))
    selected_operator = mixer_pool[max_grad_index]

    # Add new operator to the ansatz
    beta_param = Parameter(f"beta_{len(params['beta'])}")
    gamma_param = Parameter(f"gamma_{len(params['gamma'])}")

    layer = QuantumCircuit(n_qubits)
    layer.rz(gamma_param, max_grad_index)
    layer.rx(beta_param, max_grad_index)

    ansatz = ansatz.compose(layer)

    params['beta'].append(beta_param)
    params['gamma'].append(gamma_param)

    return ansatz, selected_operator

# Initialize parameters
params = {'gamma': [], 'beta': []}
ansatz = QuantumCircuit(n_qubits)
ansatz.compose(initial_state, inplace=True)

# Main ADAPT-QAOA loop
threshold = 1e-3
estimator = Estimator()

for iteration in range(10):
    ansatz, selected_op = adapt_qaoa_iteration(ansatz, mixer_pool, H_C, params, estimator)
    print(f"Iteration {iteration}: Added operator {selected_op}")

    # Check gradient norm
    gradients = [measure_gradient(op, ansatz, H_C, estimator) for op in mixer_pool]
    norm = np.linalg.norm(gradients)
    print(f"Gradient norm: {norm}")
    if norm < threshold:
        print("Convergence reached.")
        break

# Optimize the final ansatz
all_params = params['gamma'] + params['beta']
def cost_function(param_values):
    param_dict = dict(zip(all_params, param_values))
    ansatz_bound = ansatz.bind_parameters(param_dict)
    circuits = [ansatz_bound]
    result = estimator.run([H_C], circuits).result()
    energy = result.values[0]
    return np.real(energy)

initial_values = np.random.uniform(0, 2 * np.pi, len(all_params))
result = minimize(cost_function, initial_values, method='COBYLA')
print(f"Optimized energy: {result.fun}")


TypeError: Invalid circuits, expected Sequence[QuantumCircuit].